In [1]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "cfbd.settings")
import django
django.setup()

#from collections import defaultdict
import random

import pandas as pd

from main import models

In [2]:
SEASON = 2020

In [3]:
master = pd.read_csv("masterSchedule{}.csv".format(SEASON), index_col=0)
master

,ARK,ASU,AUB,BAMA,BAY,BC,CAL,CLEM,COL,CUSE,...,UTAH,UVA,VAND,VT,WAKE,WASH,WISC,WSU,WVU,ZONA
ARK,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
ASU,0,0,0,0,0,0,1,0,1,0,...,1,0,0,1,0,1,0,1,0,1
AUB,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
BAMA,1,0,1,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BAY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WASH,1,1,0,0,0,0,1,0,1,0,...,1,0,0,0,0,0,0,1,0,0
WISC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WSU,0,1,0,0,0,0,1,0,1,0,...,1,0,0,0,0,1,0,0,0,0
WVU,0,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0


## Error Checking

In [4]:
for c in master.columns:
    if master[c].sum() != 12:
        print(c, master[c].sum())

In [5]:
for r in master.index:
    if master.loc[r].sum() != 12:
        print(r, master.loc[r].sum())

## Initialize Weekly Matrix

In [6]:
weekly = pd.DataFrame([["" for i in range(1, 15+1)] for j in range(64)], columns=range(1, 15+1))
weekly.index = master.index
weekly

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
ARK,,,,,,,,,,,,,,,
ASU,,,,,,,,,,,,,,,
AUB,,,,,,,,,,,,,,,
BAMA,,,,,,,,,,,,,,,
BAY,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WASH,,,,,,,,,,,,,,,
WISC,,,,,,,,,,,,,,,
WSU,,,,,,,,,,,,,,,
WVU,,,,,,,,,,,,,,,


## Weekly Schedule

In [7]:
for i, r in weekly.iterrows():
    print(i)
    #print(r)
    s = models.School.objects.get(abbreviation=i)
    opponents = master.loc[master[i]==1,i].keys()
    print(opponents)
    
    for abb in opponents:
        
        #print(abb)
        if abb not in r.values and "@" + abb not in r.values:
            o = models.School.objects.get(abbreviation=abb)
            
            if o.conference != s.conference:  # noncon
                week = random.randint(1, 5)
                tries = 0
                while r[week] != "" or weekly.at[abb, week] != "":
                    week = random.randint(1, 5)
                    tries += 1
                    
                    if tries >= 100:
                        break
                
                if tries < 100:
                    weekly.at[i, week] = abb
                    weekly.at[abb, week] = i
                    #print(week)

                    # H/A
                    if random.randint(0,1):
                        weekly.at[i, week] = "@" + abb
                    else:
                        weekly.at[abb, week] = "@" + i
                else:
                    print("SCHEDULING ERROR:", abb)
            
            else:  # conference
                week = random.randint(4, 15)
                tries = 0
                while r[week] != "" or weekly.at[abb, week] != "":
                    week = random.randint(4, 15)
                    tries += 1
                    
                    if tries >= 100:
                        break
                
                if tries < 100:
                    weekly.at[i, week] = abb
                    weekly.at[abb, week] = i
                    #print(week)

                    # H/A
                    if random.randint(0,1):
                        weekly.at[i, week] = "@" + abb
                    else:
                        weekly.at[abb, week] = "@" + i
                else:
                    print("SCHEDULING ERROR:", abb)
        else:
            pass
            #print("already scheduled")

    #print(weekly.loc[i])
    print()
    #break

ARK
Index(['AUB', 'BAMA', 'KSU', 'LSU', 'MISS', 'MIZZ', 'MSST', 'SCAR', 'TAMU',
       'TENN', 'USC', 'WASH'],
      dtype='object')

ASU
Index(['CAL', 'COL', 'ORE', 'PITT', 'SCAR', 'UCLA', 'USC', 'UTAH', 'VT',
       'WASH', 'WSU', 'ZONA'],
      dtype='object')

AUB
Index(['ARK', 'BAMA', 'FLA', 'IND', 'KAN', 'LSU', 'MISS', 'MSST', 'TAMU',
       'TTU', 'UGA', 'VAND'],
      dtype='object')

BAMA
Index(['ARK', 'AUB', 'BC', 'CAL', 'FLA', 'KENT', 'LSU', 'MISS', 'MSST', 'PURD',
       'TAMU', 'UGA'],
      dtype='object')

BAY
Index(['ISU', 'KAN', 'KSU', 'MISS', 'OKLA', 'OKST', 'TCU', 'TEX', 'TTU',
       'UCLA', 'UNC', 'WVU'],
      dtype='object')

BC
Index(['BAMA', 'CLEM', 'CUSE', 'DUKE', 'FSU', 'ILL', 'LOU', 'MIA', 'NCST',
       'TCU', 'UNC', 'WAKE'],
      dtype='object')

CAL
Index(['ASU', 'BAMA', 'MARY', 'ORE', 'ORST', 'STAN', 'TAMU', 'UCLA', 'USC',
       'WASH', 'WSU', 'ZONA'],
      dtype='object')

CLEM
Index(['BC', 'CUSE', 'DUKE', 'FSU', 'IOWA', 'LOU', 'MIA', 'NCST', 'ORE',


In [8]:
weekly

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
ARK,@USC,@WASH,KSU,@BAMA,SCAR,,@MIZZ,TENN,@MSST,,,AUB,@TAMU,@MISS,LSU
ASU,@SCAR,VT,@PITT,@COL,,@UCLA,,@ZONA,@WASH,,CAL,UTAH,@WSU,ORE,USC
AUB,,@TTU,KAN,@UGA,@IND,@FLA,TAMU,BAMA,MISS,@LSU,,@ARK,@VAND,,MSST
BAMA,,@PURD,@CAL,ARK,@BC,@LSU,@MISS,@AUB,TAMU,,@FLA,@MSST,UGA,@KENT,
BAY,@UNC,@MISS,UCLA,TCU,,,,KAN,@OKLA,KSU,ISU,@TEX,WVU,OKST,@TTU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WASH,KAN,ARK,,@FSU,@ORST,WSU,@CAL,,ASU,ORE,@UTAH,USC,@COL,,@STAN
WISC,@ORE,@PITT,,,@MIA,PSU,@NW,,@RUTG,@OHST,@IOWA,@PURD,NEB,@ILL,@MINN
WSU,@UGA,NW,,,@ORE,@WASH,@UTAH,STAN,COL,,@ORST,,ASU,@USC,CAL
WVU,UTAH,,VT,@FLA,,TCU,,@OKLA,OKST,TEX,KAN,TTU,@BAY,ISU,@KSU


In [9]:
weekly.to_csv("weeklySchedule2020.csv")